<a href="https://colab.research.google.com/github/karenQuiroga88/python_cases/blob/main/case1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# =============================
# 🔧 CONFIGURAÇÃO INICIAL
# =============================
from google.colab import drive, auth
drive.mount('/content/drive')

import io
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import google.auth

# Autentica com sua conta Google
auth.authenticate_user()
creds, _ = google.auth.default()

# Cria cliente do Google Drive
service_drive = build('drive', 'v3', credentials=creds)

# =============================
# 📁 NOME DA PASTA NO GOOGLE DRIVE
# =============================
nome_pasta = 'Colab Notebooks'

# Busca a pasta
res = service_drive.files().list(
    q=f"name='{nome_pasta}' and mimeType='application/vnd.google-apps.folder' and trashed=false",
    spaces='drive',
    fields='files(id, name)'
).execute()

pastas = res.get('files', [])

if not pastas:
    print(f"❌ Pasta '{nome_pasta}' não encontrada.")
else:
    pasta_id = pastas[0]['id']
    print(f"✅ Pasta encontrada: {nome_pasta} (ID: {pasta_id})")

    # Lista arquivos dentro da pasta
    res_arquivos = service_drive.files().list(
        q=f"'{pasta_id}' in parents and trashed = false",
        spaces='drive',
        fields='files(id, name, mimeType)'
    ).execute()

    arquivos = res_arquivos.get('files', [])

    if not arquivos:
        print(f"📁 A pasta '{nome_pasta}' está vazia.")
    else:
        print(f"📄 {len(arquivos)} arquivos encontrados:")
        for arq in arquivos:
            print(f" - {arq['name']} ({arq['mimeType']})")

        dataframes = []

        # =============================
        # 📥 LEITURA DOS ARQUIVOS
        # =============================
        for arq in arquivos:
            nome = arq['name']
            tipo = arq['mimeType']
            print(f"\n🔹 Lendo: {nome}")

            try:
                if tipo == 'application/vnd.google-apps.spreadsheet':
                    # Exporta Google Sheets
                    request = service_drive.files().export_media(
                        fileId=arq['id'],
                        mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
                    )
                    fh = io.BytesIO()
                    downloader = MediaIoBaseDownload(fh, request)
                    done = False
                    while not done:
                        status, done = downloader.next_chunk()
                    fh.seek(0)
                    df = pd.read_excel(fh, engine='openpyxl')

                elif nome.endswith('.xlsx') or nome.endswith('.xls'):
                    # Excel
                    request = service_drive.files().get_media(fileId=arq['id'])
                    fh = io.BytesIO()
                    downloader = MediaIoBaseDownload(fh, request)
                    done = False
                    while not done:
                        status, done = downloader.next_chunk()
                    fh.seek(0)
                    df = pd.read_excel(fh, engine='openpyxl')

                elif nome.endswith('.csv'):
                    # CSV ✅
                    request = service_drive.files().get_media(fileId=arq['id'])
                    fh = io.BytesIO()
                    downloader = MediaIoBaseDownload(fh, request)
                    done = False
                    while not done:
                        status, done = downloader.next_chunk()
                    fh.seek(0)
                    df = pd.read_csv(fh)

                else:
                    print(f"⚠️ Tipo de arquivo não suportado: {nome}")
                    continue

                dataframes.append(df)
                print(f"✅ Sucesso: {nome}")
            except Exception as e:
                print(f"⚠️ Erro ao ler '{nome}': {e}")

        # =============================
        # 🔗 UNIR TODOS OS DATAFRAMES
        # =============================
        if dataframes:
            df_final = pd.concat(dataframes, ignore_index=True)
            print("\n✅ DataFrame final criado com sucesso!")
            print(df_final.head())
        else:
            print("\n⚠️ Nenhum arquivo pôde ser lido.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Pasta encontrada: Colab Notebooks (ID: 1hYFZL3AQxREHg8yuUDmSTU4jddAtrTZw)
📄 1 arquivos encontrados:
 - ScreenTimevsMentalWellness.csv (text/csv)

🔹 Lendo: ScreenTimevsMentalWellness.csv
✅ Sucesso: ScreenTimevsMentalWellness.csv

✅ DataFrame final criado com sucesso!
  user_id  age  gender occupation  work_mode  screen_time_hours  \
0   U0001   33  Female   Employed     Remote              10.79   
1   U0002   28  Female   Employed  In-person               7.40   
2   U0003   35  Female   Employed     Hybrid               9.78   
3   U0004   42    Male   Employed     Hybrid              11.13   
4   U0005   28    Male    Student     Remote              13.22   

   work_screen_hours  leisure_screen_hours  sleep_hours  sleep_quality_1_5  \
0               5.44                  5.35         6.63                  1   
1               0.37                  7.03 